In [9]:
import json
file_name = "Books_small_10000.json"
with open(file_name) as f:
    for line in f:
        review = json.loads(line)
        print(review["reviewText"])
        print(review["overall"])
        break

I bought both boxed sets, books 1-5.  Really a great series!  Start book 1 three weeks ago and just finished book 5.  Sloane Monroe is a great character and being able to follow her through both private life and her PI life gets a reader very involved!  Although clues may be right in front of the reader, there are twists and turns that keep one guessing until the last page!  These are books you won't be disappointed with.
5.0


In [65]:
import random

In [180]:
class Sentiment:
    NEGATIVE = "NEGATIVE"
    NEUTRAL = "NEUTRAL"
    POSITIVE = "POSITIVE"

class Review:
    def __init__(self,text,score):
        self.text = text
        self.score = score
        self.sentiment = self.get_sentiment()
        
    def get_sentiment(self):
        if self.score <= 2:
            return Sentiment.NEGATIVE
        elif self.score== 3:
            return Sentiment.NEUTRAL
        else:
            return Sentiment.POSITIVE

"""class ReviewContainer:
    def __init__(self,reviews):
        self.reviews = reviews
    def evenly_distribute(self):
        negative = list([x for x in self.reviews if x== Sentiment.NEGATIVE])
        positive = list([x for x in self.reviews if x== Sentiment.POSITIVE])
        print(len(negative))
        print(len(positive))"""

class ReviewContainer:
    def __init__(self,reviews):
        self.reviews = reviews
    def get_text(self):
        return [x.text for x in self.reviews]
    def get_sentiment(self):
        return [x.sentiment for x in self.reviews]

        
    def evenly_distribute(self):
        negative = list(filter(lambda x: x.sentiment == Sentiment.NEGATIVE,self.reviews))
        positive = list(filter(lambda x: x.sentiment == Sentiment.POSITIVE,self.reviews))
        positive_shrunk = positive[:len(negative)]
        self.reviews = negative + positive_shrunk
        random.shuffle(self.reviews)
        
        


In [181]:
import json

file_name = "Books_small_10000.json"
reviews = []
with open(file_name) as f:
    for line in f:
        review = json.loads(line)
        reviews.append(Review(review["reviewText"],review["overall"]))


In [162]:
reviews[5].sentiment

'POSITIVE'

### Prep Data

In [182]:
from sklearn.model_selection import train_test_split


In [183]:
training, test = train_test_split(reviews,test_size = 0.33,random_state = 42)

In [202]:
train_container = ReviewContainer(training)
test_container = ReviewContainer(test)

train_container.evenly_distribute()
test_container.evenly_distribute()


In [212]:
train_x = train_container.get_text()
train_y = train_container.get_sentiment()

test_x = test_container.get_text()
test_y = test_container.get_sentiment()

In [219]:
print(train_y.count(Sentiment.NEGATIVE))
print(train_y.count(Sentiment.POSITIVE))

436
436


In [220]:
print(test_y.count(Sentiment.NEGATIVE))
print(test_y.count(Sentiment.POSITIVE))

208
208


### Bags of Words Vectorization

In [221]:
from sklearn.feature_extraction.text import CountVectorizer

In [222]:
vectorizer = CountVectorizer()
train_x_vectors=vectorizer.fit_transform(train_x)
test_x_vectors=vectorizer.transform(test_x)

In [223]:
print(train_x[0])
print(train_x_vectors[0])

So thrilled to see this is finally available for the Kindle. I have read this series so many times my paperback copies are in tatters. This was one of the first fantasy series I read and it holds a very special place in my heart. The story is well told and the characters are fun to follow.
<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 40 stored elements and shape (1, 8906)>
  Coords	Values
  (0, 7280)	2
  (0, 8000)	1
  (0, 8052)	2
  (0, 6955)	1
  (0, 7976)	3
  (0, 4264)	2
  (0, 3065)	1
  (0, 667)	1
  (0, 3177)	1
  (0, 7929)	4
  (0, 4476)	1
  (0, 3669)	1
  (0, 6392)	2
  (0, 7002)	2
  (0, 4891)	1
  (0, 8039)	1
  (0, 5260)	2
  (0, 5685)	1
  (0, 1761)	1
  (0, 525)	2
  (0, 4034)	2
  (0, 7817)	1
  (0, 8608)	1
  (0, 5514)	1
  (0, 5478)	1
  (0, 3087)	1
  (0, 2961)	1
  (0, 423)	2
  (0, 4277)	1
  (0, 3813)	1
  (0, 8497)	1
  (0, 7361)	1
  (0, 5900)	1
  (0, 3697)	1
  (0, 7533)	1
  (0, 8665)	1
  (0, 8063)	1
  (0, 1362)	1
  (0, 3284)	1
  (0, 3162)	1


# Classification

### Linear SVM

In [224]:
from sklearn import svm

In [225]:
clf_svm = svm.SVC(kernel = "linear")
clf_svm.fit(train_x_vectors,train_y)

,C,1.0
,kernel,'linear'
,degree,3
,gamma,'scale'
,coef0,0.0
,shrinking,True
,probability,False
,tol,0.001
,cache_size,200
,class_weight,None
,verbose,False


In [226]:
test_x[10]
clf_svm.predict(test_x_vectors[10])

array(['NEGATIVE'], dtype='<U8')

### Decision Trees

In [20]:
from sklearn.tree import DecisionTreeClassifier
clf_dec = DecisionTreeClassifier()
clf_dec.fit(train_x_vectors,train_y)
clf_dec.predict(test_x_vectors[10])

array(['POSITIVE'], dtype='<U8')

### Naive Bayes

In [18]:
from sklearn.naive_bayes import MultinomialNB

clf_nb = MultinomialNB()
clf_nb.fit(train_x_vectors, train_y)
clf_nb.predict(test_x_vectors[10])


array(['POSITIVE'], dtype='<U8')

#### Logistic Regression

In [17]:
from sklearn.linear_model import LogisticRegression

clf_log = LogisticRegression()
clf_log.fit(train_x_vectors, train_y)
clf_log.predict(test_x_vectors[10])

C:\Users\aditi\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


array(['POSITIVE'], dtype='<U8')

# Evaluation

In [21]:
# Mean accuracy
print(clf_svm.score(test_x_vectors,test_y))
print(clf_dec.score(test_x_vectors,test_y))
print(clf_nb.score(test_x_vectors,test_y))
print(clf_log.score(test_x_vectors,test_y))

0.8124242424242424
0.7681818181818182
0.8369696969696969
0.8409090909090909


In [22]:
# F1 Score
from sklearn.metrics import f1_score

f1_score(test_y, clf_svm.predict(test_x_vectors),average = None, labels = ["POSITIVE","NEGATIVE","NEUTRAL"])

array([0.90738061, 0.40268456, 0.2656    ])

In [25]:
train_y.count("NEGATIVE")

436